In [1]:
%matplotlib inline

In [2]:
import deeplabcut as dlc
import numpy as np
import os 
import glob
import imageio
import itertools

C:\Users\Peter\anaconda3\envs\behavior_analysis\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Peter\anaconda3\envs\behavior_analysis\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Peter\anaconda3\envs\behavior_analysis\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Peter\anaconda3\envs\behavio

### Helper functions

In [3]:
def get_files(source, wcs=['_FRONT.avi']):
    matches = []
    for root, dirnames, filenames in os.walk(source):
        for filename in filenames:
            if filename.endswith(tuple(wcs)):
                matches.append(os.path.join(root, filename))
    return matches

In [4]:
def print_dict(ddict, length=True):
    for k, v in ddict.items():
        if length:
            print(f'{k}: {len(v)}')
        else:
            print(f'{k}: {v}')

### Load images 

In [5]:
parent_directory = r'C:\Users\Peter\Desktop\DLC'
project_directory = os.path.join(parent_directory, 'test')

In [ ]:
image_master_dir = os.path.join(parent_directory, 'M7')
image_dirs = [os.path.join(image_master_dir, x) for x in os.listdir(image_master_dir)]
image_dirs = [x for x in image_dirs if os.path.isdir(x)]
dict_of_avis = {x: get_files(x) for x in image_dirs}
print_dict(dict_of_avis)

Select subset of images to train. Make sure to include empty pellets, successes, failures 

In [ ]:
selection_dict = {
    r'C:\Users\Peter\Desktop\DLC\M7\2021-02-16': list(range(20, 30)),
    r'C:\Users\Peter\Desktop\DLC\M7\2021-02-17': list(range(60, 70)),
    r'C:\Users\Peter\Desktop\DLC\M7\2021-02-20': list(range(0, 10)),
    r'C:\Users\Peter\Desktop\DLC\M7\2021-02-21': list(range(30, 40))
                                                      }

In [ ]:
image_master_dir = os.path.join(parent_directory, 'M7')
image_dirs = [os.path.join(image_master_dir, x) for x in os.listdir(image_master_dir)]
image_dirs = [x for x in image_dirs if os.path.isdir(x)]
dict_of_avis = {x: get_files(x) for x in image_dirs}
for k, v in dict_of_avis.items():
    v = np.array(v)
    dict_of_avis[k] = list(v[selection_dict[k]])
print_dict(dict_of_avis, length=True)

In [ ]:
list_of_avis = list(itertools.chain(*dict_of_avis.values()))
print(len(list_of_avis))

### Create project

In [6]:
# DO ONCE
# config_path = dlc.create_new_project('test', 
#                                      'pw', 
#                                      videos=list_of_avis)
project_path = r'C:\Users\Peter\Desktop\DLC\test-pw-2021-02-24'
config_path = os.path.join(project_path, 'config.yaml')

In [ ]:
dlc.extract_frames(config_path, 
                  mode='manual',
                  algo='kmeans',
                  crop=True,
                  userfeedback=False)

In [ ]:
dlc.label_frames(config_path)

### Train

In [ ]:
dlc.create_training_dataset(config_path, augmenter_type='imgaug')

In [ ]:
dlc.train_network(config_path, displayiters=1000, saveiters=10000, )

### Plot

In [8]:
video_path = os.path.join(project_path, 'videos_dgp')
dest_folder = video_path + '_dlc'

In [11]:
dlc.analyze_videos(config_path, 
                          videos=[video_path], 
                          videotype='avi',
                  destfolder=dest_folder)
# dlc.filterpredictions(config_path, [video_path], videotype='avi')
dlc.create_labeled_video(config_path, [video_path], videotype='avi', filtered=False, destfolder=dest_folder)

Using snapshot-920000 for model C:\Users\Peter\Desktop\DLC\test-pw-2021-02-24\dlc-models\iteration-1\testFeb24-trainset85shuffle1
Initializing ResNet
Starting to analyze %  C:\Users\Peter\Desktop\DLC\test-pw-2021-02-24\videos_dgp/M7_2021-02-16_00022_FRONT.avi
C:\Users\Peter\Desktop\DLC\test-pw-2021-02-24\videos_dgp_dlc  already exists!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Analyzing all the videos in the directory...


KeyboardInterrupt: 

### Refine

In [11]:
dlc.extract_outlier_frames(config_path, 
                           [dest_folder + r'\\M7_2021-02-16_00027_FRONT.avi'],
                           outlieralgorithm='manual')

Creating the symbolic link of the video


OSError: symbolic link privilege not held

In [ ]:
dlc.refine_labels(config_path)

In [ ]:
dlc.merge_datasets(config_path)

In [9]:
dest_folder + r'/M7_2021-02-16_00022_FRONT.avi'

'C:\\Users\\Peter\\Desktop\\DLC\\test-pw-2021-02-24\\videos_dgp_dlc/M7_2021-02-16_00022_FRONT.avi'